In [2]:
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["SERPAPI_API_KEY"] = ""

In [3]:
from langchain import OpenAI, SerpAPIWrapper
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

llm = OpenAI(temperature=0)

### SerpAPI
* A search engine. Useful for when you need to answer questions about current events.

### Tool
> The ‘Tool’ dataclass wraps functions that accept a single string input and returns a string output.

   #### params:
* name: a unique name given to each tool, has to be named intermediate answer in this example
``` python
 @property def observation_prefix(self) -> str:
    """Prefix to append the observation with."""
    return "Intermediate answer
```: "

In [4]:
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description="useful for when you need to ask with search"
    )
]

# tools = []

In [5]:
self_ask_with_search = initialize_agent(tools, llm, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True, return_intermediate_steps=True)
question = "What is the hometown of the reigning men's U.S. Open champion?"

In [7]:
response = self_ask_with_search({"input":question})



> Entering new AgentExecutor chain...
 Yes.
Follow up: Who is the reigning men's U.S. Open champion?
Intermediate answer: US OpenEdit ; 1984. USA, John McEnroe ; 1985 · TCH · Ivan Lendl ; 1986 · TCH · Ivan Lendl ; 1987 · TCH · Ivan Lendl.
Follow up: Where is John McEnroe from?
Intermediate answer: Wiesbaden, Germany
So the final answer is: Wiesbaden, Germany

> Finished chain.


## whether or not call intermediate step

`_call`
```python
while self._should_continue(iterations, time_elapsed):
        next_step_output = self._take_next_step(
            name_to_tool_map, color_mapping, inputs, intermediate_steps
        )
        if isinstance(next_step_output, AgentFinish):
            return self._return(next_step_output, intermediate_steps)
```

`_take_next_step`
* if `return = AGENT_FINISH` 則FINISH
* if `return = AGENT_ACTION` 則使用TOOLS
* 詢問llm要return什麼東西

### Scratchpad
```python
def _construct_scratchpad(
        self, intermediate_steps: List[Tuple[AgentAction, str]]
    ) -> Union[str, List[BaseMessage]]:
        """Construct the scratchpad that lets the agent continue its thought process."""
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\n{self.observation_prefix}{observation}\n{self.llm_prefix}"
        return thoughts
```

In [10]:
response

{'input': "What is the hometown of the reigning men's U.S. Open champion?",
 'output': 'Wiesbaden, Germany',
 'intermediate_steps': [(AgentAction(tool='Intermediate Answer', tool_input="Who is the reigning men's U.S. Open champion?", log=" Yes.\nFollow up: Who is the reigning men's U.S. Open champion?"),
   'US OpenEdit ; 1984. USA, John McEnroe ; 1985 · TCH · Ivan Lendl ; 1986 · TCH · Ivan Lendl ; 1987 · TCH · Ivan Lendl.'),
  (AgentAction(tool='Intermediate Answer', tool_input='Where is John McEnroe from?', log='Follow up: Where is John McEnroe from?'),
   'Wiesbaden, Germany')]}

In [11]:
response['intermediate_steps']

[(AgentAction(tool='Intermediate Answer', tool_input="Who is the reigning men's U.S. Open champion?", log=" Yes.\nFollow up: Who is the reigning men's U.S. Open champion?"),
  'US OpenEdit ; 1984. USA, John McEnroe ; 1985 · TCH · Ivan Lendl ; 1986 · TCH · Ivan Lendl ; 1987 · TCH · Ivan Lendl.'),
 (AgentAction(tool='Intermediate Answer', tool_input='Where is John McEnroe from?', log='Follow up: Where is John McEnroe from?'),
  'Wiesbaden, Germany')]

### Alternatives

In [15]:
from langchain import SelfAskWithSearchChain

In [17]:
self_ask = SelfAskWithSearchChain(llm=llm, search_chain=search, verbose=True, return_intermediate_steps=True)

In [18]:
response1 = self_ask(question)



> Entering new SelfAskWithSearchChain chain...
 Yes.
Follow up: Who is the reigning men's U.S. Open champion?
Intermediate answer: US OpenEdit ; 1984. USA, John McEnroe ; 1985 · TCH · Ivan Lendl ; 1986 · TCH · Ivan Lendl ; 1987 · TCH · Ivan Lendl.
Follow up: Where is John McEnroe from?
Intermediate answer: Wiesbaden, Germany
So the final answer is: Wiesbaden, Germany

> Finished chain.


In [19]:
response1

{'input': "What is the hometown of the reigning men's U.S. Open champion?",
 'output': 'Wiesbaden, Germany',
 'intermediate_steps': [(AgentAction(tool='Intermediate Answer', tool_input="Who is the reigning men's U.S. Open champion?", log=" Yes.\nFollow up: Who is the reigning men's U.S. Open champion?"),
   'US OpenEdit ; 1984. USA, John McEnroe ; 1985 · TCH · Ivan Lendl ; 1986 · TCH · Ivan Lendl ; 1987 · TCH · Ivan Lendl.'),
  (AgentAction(tool='Intermediate Answer', tool_input='Where is John McEnroe from?', log='Follow up: Where is John McEnroe from?'),
   'Wiesbaden, Germany')]}